In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

from goatools.base import download_go_basic_obo, download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader

## Load Ontologies, Associations and Background gene set 

### Ontologies

下載下來之後，我們使用```goatools.obo_parser.GODag```把GO terms處理成 GO_ID: GOTerm 的形式。 

In [2]:
# Get http://geneontology.org/ontology/go-basic.obo
obo_fname = download_go_basic_obo()
obodag = GODag("go-basic.obo")

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2020-05-02) 47,240 GO Terms


### Associations

下載之後，用```Gene2GoReader```把association處理成 NCBI GeneID: [GO_IDs]

In [3]:
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
fin_gene2go = download_ncbi_associations()

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[9606])

# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated genes".format(NS=nspc, N=len(id2gos)))

  EXISTS: gene2go
HMS:0:00:05.117067 336,356 annotations, 20,586 genes, 18,410 GOs, 1 taxids READ: gene2go 
MF 17,580 annotated genes
CC 19,399 annotated genes
BP 18,607 annotated genes


### Background gene set

```goatools```有可以直接load的函數

In [4]:
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT as GeneID2nt_human

In [5]:
len(GeneID2nt_human.keys())

20913

### Our study genes

整理成```{gene id: gene sybmol}```的形式。

In [6]:
geneid2symbol = {}
with open("over-represented_mutated_genes.txt", "r") as f:
    for line in f:
        symbol, geneid = line.rstrip().split("\t")
        if geneid != "None":
            geneid2symbol[int(geneid)] = symbol

-----

## Run Gene Ontology Enrichment Analysis (GOEA)

* 這個 GOEA 的物件包含 Ontologies, Associations, 和 background gene sets。初始化之後可以重複使用。

In [7]:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

goeaobj = GOEnrichmentStudyNS(
    GeneID2nt_human.keys(), # List of human protein-coding genes
    ns2assoc, # geneid/GO associations
    obodag, # Ontologies
    propagate_counts = False,
    alpha = 0.05, # default significance cut-off
    methods = ['fdr_bh']  # defult multipletest correction method
)


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 81% 16,885 of 20,913 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 85% 17,850 of 20,913 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 80% 16,792 of 20,913 population items found in association


真的執行GSEA的演算法，並回傳結果。

In [8]:
geneids_study = geneid2symbol.keys()
goea_results_all = goeaobj.run_study(geneids_study)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run BP Gene Ontology Analysis: current study set of 317 IDs ... 95%    300 of    317 study items found in association
100%    317 of    317 study items found in population(20913)
Calculating 12,291 uncorrected p-values using fisher_scipy_stats
  12,291 GO terms are associated with 16,885 of 20,913 population items
   1,904 GO terms are associated with    300 of    317 study items
  METHOD fdr_bh:
      36 GO terms found significant (< 0.05=alpha) ( 36 enriched +   0 purified): statsmodels fdr_bh
     154 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 317 IDs ... 97%    307 of    317 study items found in association
100%    317 of    317 study items found in population(20913)
Calculating 1,754 uncorrected p-values using fisher_scipy_stats
   1,754 GO terms are associated with 17,850 of 20,913 population items
     405 GO terms are associated with    307 of 

In [10]:
from collections import Counter

ctr = Counter([r.NS for r in goea_results_sig])
print('Significant results[{TOTAL}] = {BP} BP + {MF} MF + {CC} CC'.format(
    TOTAL=len(goea_results_sig),
    BP=ctr['BP'],  # biological_process
    MF=ctr['MF'],  # molecular_function
    CC=ctr['CC'])  # cellular_component
)

Significant results[92] = 36 BP + 16 MF + 40 CC


## Write results to a file

In [37]:
goeaobj.wr_xlsx("ORA_results.xlsx", goea_results_sig)
goeaobj.wr_txt("ORA_results.txt", goea_results_sig)
goeaobj.wr_tsv("ORA_results.tsv", goea_results_sig)

     92 items WROTE: ORA_results.xlsx
     92 GOEA results for   248 study items. WROTE: ORA_results.txt


In [12]:
len(goea_results_sig), type(goea_results_sig)

(92, list)

## Manual Over-representation Analysis

* 為了了解ORA背後的實際在做什麼，從源頭練習一次學最快~

利用上面讀取的 gene id: GO Term 轉換成 GO Term: GO ID

In [13]:

bp_term2id = {}

for idx, terms in ns2assoc["BP"].items():
    for t in terms:
        if t in bp_term2id:
            bp_term2id[t].append(idx)
        else:
            bp_term2id[t] = [idx]

使用單一一個Gene Set

In [14]:
target_term = "GO:0007155"

In [15]:
gene_id_in_term = [idx for idx, terms in ns2assoc["BP"].items() if target_term in terms]

In [16]:
query_list = list(geneid2symbol.keys())

In [17]:
k = len(set(gene_id_in_term).intersection(set(query_list)))
m = len(query_list)  # Query set
n = len(gene_id_in_term)  # Target Gene set
N = len(GeneID2nt_human.keys())  # Background set

In [18]:
from scipy.stats import fisher_exact

table = np.array([
    [k, n - k],
    [m - k, N + k - n - m]
])

oddsratio, pval = fisher_exact(table)

In [19]:
pval

1.2873180658437553e-14

如果要對每個Gene Set都做，這時候就可以自己定義一個函數

In [20]:
def manual_ORA(query_set, gene_set):

    k = len(gene_set.intersection(query_set))
    m = len(query_set)  # Query set
    n = len(gene_set)  # Target Gene set
    N = len(GeneID2nt_human.keys())  # Background set

    table = np.array([
        [k, n - k],
        [m - k, N + k - n - m]
    ])

    oddsratio, pval = fisher_exact(table)
    return pval

把每個 GO term 和相對應的 Gene Set 一個一個放進 ```manual_ORA``` ，並記錄回傳的p-value

In [78]:
query_set = set(query_list)
pvalues = {}
for term, ids in bp_term2id.items():
    gene_set = set(ids)
    pval = manual_ORA(query_set, gene_set)
    pvalues[term] = pval

這裏是用去校正p-values以得到比較嚴格的p-value數值。

In [93]:
from statsmodels.stats.multitest import multipletests
flat_pvalues = [(term, p) for term, p in pvalues.items()]
terms = [x[0] for x in flat_pvalues]
pvals = [x[1] for x in flat_pvalues]

_, pvals, _, _ = multipletests(pvals, method="fdr_bh", alpha=0.05)

In [95]:
manual_sig = {term: p for term, p in zip(terms, pvals) if p < 0.05}

In [102]:
[obodag[term].name for term in manual_sig.keys()]

['G protein-coupled receptor signaling pathway',
 'cell adhesion',
 'modulation of chemical synaptic transmission',
 'ion transmembrane transport',
 'positive regulation of synapse assembly',
 'protein localization to synapse',
 'synapse assembly',
 'nervous system development',
 'axonogenesis',
 'regulation of NMDA receptor activity',
 'positive regulation of potassium ion transmembrane transporter activity',
 'cell-cell adhesion',
 'positive regulation of synaptic transmission, glutamatergic',
 'collagen fibril organization',
 'heterophilic cell-cell adhesion via plasma membrane cell adhesion molecules',
 'adult behavior',
 'regulation of presynapse assembly',
 'learning',
 'positive regulation of axonogenesis',
 'calcium-dependent cell-cell adhesion via plasma membrane cell adhesion molecules',
 'cell-cell junction assembly',
 'neuron cell-cell adhesion',
 'homophilic cell adhesion via plasma membrane adhesion molecules',
 'positive regulation of excitatory postsynaptic potential',


## Practices

* 將manual_ORA的結果整理成表格輸出成csv檔案。
* 比較manual_ORA跟goatools的結果。

## References

* Example adopted from https://github.com/tanghaibao/goatools/blob/master/notebooks/goea_nbt3102.ipynb